In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import json

In [547]:
# Đọc dữ liệu đã xử lý
data = pd.read_csv('../data/processed/diamonds_clean.csv')

# Đọc thông tin folds
with open('../data/kfold_indices.json', 'r') as f:
    fold_indices = json.load(f)

In [549]:
# Giả sử cột target là 'price'
X = data.drop(columns=['price'])
y = data['price']

print("Shape dữ liệu:", X.shape, y.shape)

Shape dữ liệu: (46425, 9) (46425,)


In [550]:
# create log dataset
log_data = data.copy()
log_data['log_price'] = np.log(log_data['price'])
log_data['log_carat'] = np.log(log_data['carat'])

In [551]:
def cross_val_mae_sklearn(X, y, model, k=5, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    maes = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Sklearn tự động xử lý intercept (hệ số chặn)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        # If this is Model 5 using log target, convert back to original scale
        if ('log_carat' in X_val.columns) and (getattr(y_val, 'name', None) == 'log_price'):
            y_pred = np.exp(y_pred)
            y_val = np.exp(y_val)
        
        # Sử dụng MAE của sklearn
        maes.append(mean_absolute_error(y_val, y_pred))
    
    return np.mean(maes)

In [552]:
model_definitions = [
    # Minh
    {"Tên mô hình": "Model 1: carat", 
     "Đặc trưng": ['carat']},

    # My
    {"Tên mô hình": "Model 2: log(carat) + color + clarity + x*y/z", 
     "Đặc trưng": ['log_carat', 'color', 'clarity', 'x*y/z']},

    # Kiệt
    {"Tên mô hình": "Model 3: carat + x + y + z", 
     "Đặc trưng": ['carat', 'x', 'y', 'z']},

    # Thảo 
    {"Tên mô hình": "Model 4: carat + depth + table + x + y + z", 
     "Đặc trưng": ['carat', 'depth', 'table', 'x', 'y', 'z']},

    # Duy - log-linear regression - tranform carat and price using log 
    {"Tên mô hình": "Model 5: log(carat) + cut + color + clarity", 
     "Đặc trưng": ['log_carat', 'cut', 'color', 'clarity']},

    # Kiệt thêm mô hình mới
    {"Tên mô hình": "Model 6: carat + volume",
    "Đặc trưng": ['carat', 'volume']}
]

In [553]:
results = []

for m in model_definitions:
    cols = m["Đặc trưng"]
    
    # Khởi tạo mô hình sklearn
    model = LinearRegression()
    if (m["Tên mô hình"] == "Model 2: log(carat) + color + clarity + x*y/z"):
        X['x*y/z'] = data['x'] * data['y'] / data['z']
        X['log_carat'] = log_data['log_carat']
        y = log_data['log_price']

    if(m["Tên mô hình"] == "Model 6: carat + volume"):
        if "volume" not in X.columns:
            X["volume"] = data["x"] * data["y"] * data["z"]

    if (m["Tên mô hình"] == "Model 5: log(carat) + cut + color + clarity"):
        X[cols] = log_data[cols]
        y = log_data['log_price']
    mae_score = cross_val_mae_sklearn(X[cols], y, model, k=5)
    
    y = data['price']
    
    print(f"Hoàn thành: {m['Tên mô hình']}")
    
    results.append({
        "Tên mô hình": m["Tên mô hình"],
        "Đặc trưng": cols,
        "MAE trung bình": mae_score
    })

Hoàn thành: Model 1: carat
Hoàn thành: Model 2: log(carat) + color + clarity + x*y/z
Hoàn thành: Model 3: carat + x + y + z
Hoàn thành: Model 4: carat + depth + table + x + y + z
Hoàn thành: Model 5: log(carat) + cut + color + clarity
Hoàn thành: Model 6: carat + volume


In [554]:
results_df = pd.DataFrame(results)
results_df.sort_values("MAE trung bình")

,Tên mô hình,Đặc trưng,MAE trung bình
4,Model 5: log(carat) + cut + color + clarity,"[log_carat, cut, color, clarity]",324.591959
1,Model 2: log(carat) + color + clarity + x*y/z,"[log_carat, color, clarity, x*y/z]",330.142580
3,Model 4: carat + depth + table + x + y + z,"[carat, depth, table, x, y, z]",600.706672
2,Model 3: carat + x + y + z,"[carat, x, y, z]",605.422968
5,Model 6: carat + volume,"[carat, volume]",646.281232
0,Model 1: carat,[carat],648.584184
